In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
testing_df = pd.read_csv('../data/test_set_basic.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,renal_biopsy_class,anti_cardioliphin_antibodies,anti_b2gp1_antibodies,lupus_anti_coagulant,c3,c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,5,0,0,0,1,1,1,0,1
3,1,0,0,0,0,1,0,0,0,1,...,0,2,0,0,0,1,1,0,0,1
4,1,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,1,0,0,0


In [9]:
len(testing_df[testing_df.ana == 0])

3483

In [4]:
testing_df.tail()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,renal_biopsy_class,anti_cardioliphin_antibodies,anti_b2gp1_antibodies,lupus_anti_coagulant,c3,c4,anti_dsdna_antibody,anti_smith_antibody,label
13995,0,1,0,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
13996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
13997,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
13998,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
13999,1,0,0,1,0,1,0,0,0,0,...,0,1,1,1,0,1,0,0,0,1


In [5]:
X_test = testing_df.iloc[:, 0:-1]
y_test = testing_df.iloc[:, -1]

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 26), (14000,))

In [6]:
dqn_model = utils.load_dqn3('../models/dqn_unbalanced_14000000')
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
test_df.head()

Using stable baselines 3
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,score,is_success
0,0.0,4.0,1.0,0.0,0.0,"[ana, renal_biopsy_class, pericardial_effusion...",0.0,0.0,1.0
1,1.0,4.0,1.0,0.0,0.0,"[ana, renal_biopsy_class, pericardial_effusion...",False,0.0,True
2,2.0,8.0,1.0,1.0,1.0,"[ana, renal_biopsy_class, anti_cardioliphin_an...",False,0.0,True
3,3.0,8.0,1.0,1.0,1.0,"[ana, renal_biopsy_class, anti_cardioliphin_an...",False,0.0,True
4,4.0,6.0,1.0,0.0,0.0,"[ana, renal_biopsy_class, acute_pericarditis, ...",False,0.0,True


In [11]:
len(test_df[test_df.episode_length==4])

2943

In [ ]:
test_df.episode_length.min(), test_df.episode_length.max()

In [ ]:
test_df[test_df.episode_length == 4]

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
acc, precision, recall, f1 = utils.test_binary(test_df['y_actual'], test_df['y_pred'])
acc, precision, recall, roc_auc

In [ ]:
test_df.y_pred.value_counts()

In [ ]:
test_df.y_actual.value_counts()

#### Saving files

In [ ]:
test_df.to_csv(f'../test_dfs/dqn_test_df_balanced_14000000.csv', index=False)
success_df.to_csv(f'../test_dfs/dqn_success_df_balanced_14000000.csv', index=False)